In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# Comparacion de Resultados GRD y DEIS

En este cuadernillo se quieren calcular las diferencias entre las bases de egresos hospitalarios GRD y DEIS. Se compararán:

1. Cantidad de egresos hospitalarios
2. Ranking para cada uno de los diagnósticos

Este análisis se realizará a nivel Nacional y a nivel del Hospital del Tórax

In [18]:
# Lee ranking de egresos DEIS
ranking_egresos_nacionales_deis = pl.read_csv(
    "../data/external/ranking_nacional_egresos.csv", encoding="latin-1", separator=";"
).to_pandas()

# Lee ranking de egresos GRD
ranking_egresos_nacionales_grd = pl.read_csv(
    "../data/interim/ranking_nacionales_egresos_grd.csv", separator=";"
).to_pandas()

In [25]:
# Define columnas a analizar entre ambas bases
COLUMNAS_A_SELECCIONAR = [
    "ANIO_EGRESO",
    "COD_HOSPITAL",
    "DIAGNOSTICO1",
    "Capítulo",
    "Sección",
    "Categoría",
    "Descripción",
    "GLOSA_ESTABLECIMIENTO_SALUD",
    "n_egresos_GRD",
    "n_egresos_DEIS",
    "ranking_egresos",
    "ranking_grd_n_egresos",
    "dias_estada_totales_GRD",
    "dias_estada_totales_DEIS",
    "ranking_nacionales_n_egresos",
    "ranking_publicos_n_egresos",
]

# Une ambas bases de datos segun el anio, hospital y diagnostico principal
comparacion_grd_deis_pais = (
    ranking_egresos_nacionales_grd.merge(
        ranking_egresos_nacionales_deis,
        left_on=["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"],
        right_on=["ANO_EGRESO", "ESTABLECIMIENTO_SALUD", "DIAG1"],
        how="inner",
        suffixes=("_GRD", "_DEIS"),
    )[COLUMNAS_A_SELECCIONAR]
)

# Obtiene comparacion de egresos solamente para el Torax
comparacion_grd_deis_torax = comparacion_grd_deis_pais.query("COD_HOSPITAL == 112103").copy()

In [26]:
DIFERENCIAS_A_CALCULAR = [
    ("n_egresos_GRD", "n_egresos_DEIS"),
    ("ranking_egresos", "ranking_grd_n_egresos"),
]

# Calcula diferencias entre egresos y rankings a nivel pais y Torax
comparacion_grd_deis_torax = build_features.calculate_and_add_difference(
    comparacion_grd_deis_torax, DIFERENCIAS_A_CALCULAR
)

comparacion_grd_deis_pais = build_features.calculate_and_add_difference(
    comparacion_grd_deis_pais, DIFERENCIAS_A_CALCULAR
)

In [28]:
# Obtiene el promedio de las diferencias de egresos
diferencia_promedio_egresos_torax = comparacion_grd_deis_torax[
    "n_egresos_GRD_n_egresos_DEIS_difference"
].mean()

# Obtiene el promedio de las diferencias de rankings
diferencia_promedio_ranking_torax = comparacion_grd_deis_torax[
    "ranking_egresos_ranking_grd_n_egresos_difference"
].mean()

print(f"> {diferencia_promedio_egresos_torax=:.2f} egresos")
print(f"> {diferencia_promedio_ranking_torax=:.2f} posiciones")

> diferencia_promedio_egresos_torax=3.08 egresos
> diferencia_promedio_ranking_torax=4.46 posiciones


In [29]:
# Obtiene el promedio de las diferencias de egresos
diferencia_promedio_egresos_pais = comparacion_grd_deis_pais[
    "n_egresos_GRD_n_egresos_DEIS_difference"
].mean()

# Obtiene el promedio de las diferencias de rankings
diferencia_promedio_ranking_pais = comparacion_grd_deis_pais[
    "ranking_egresos_ranking_grd_n_egresos_difference"
].mean()

print(f"> {diferencia_promedio_egresos_pais=:.2f} egresos")
print(f"> {diferencia_promedio_ranking_pais=:.2f} posiciones")

> diferencia_promedio_egresos_pais=6.95 egresos
> diferencia_promedio_ranking_pais=6.14 posiciones


# Diferencias en cantidad de Hospitales presentes